In [1]:
#Generate Pathways using ProcessedRels with Thermo data

from openbabel import openbabel
from rdkit import Chem
from rdkit.Chem import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
from treelib import Node, Tree
from ast import literal_eval
import time
from func_timeout import *
import signal
from contextlib import contextmanager
import time

In [2]:
import os
os.environ['kmp_duplicate_lib_ok']='true'

In [3]:
class Molecule(object):
    def __init__(self, path):
        self.path = path
def split(word):
    return[char for char in word]
def tree_copier(tree, identifier_factor):
    tree_copy = Tree()
    nodes = tree.all_nodes()
    for i in range(len(nodes)):
        dummy_tag = nodes[i].tag
        dummy_identifier = nodes[i].identifier
        dummy_data = nodes[i].data
        try:
            dummy_parent = (tree.parent(dummy_identifier)).identifier 
        except:
            dummy_parent = -1
        if dummy_parent == -1:
            tree_copy.create_node(dummy_tag, (dummy_identifier+identifier_factor), data = dummy_data)
        else:
            tree_copy.create_node(dummy_tag, (dummy_identifier+identifier_factor), parent=(dummy_parent+identifier_factor), data = dummy_data)
    return(tree_copy)

In [4]:
def index_finder(Product, rels, path):
    indexes = []
    for i in range(len(rels['Index'])):
        if rels['Energy Change'][i] != 'NaN':
            place = literal_eval(rels['Products'][i])
            for j in range(len(place)):
                if place[j] == Product:
                    indexes.append(rels['Index'][i])
    valid_indexes = []
    for i in range(len(indexes)):
        valid = True
        precursors = precursor_finder(indexes[i], rels)
        for j in range(len(precursors)):
            if precursors[j] in path:
                valid = False
                break
        if valid == True:
            valid_indexes.append(indexes[i])
    return(valid_indexes)
def precursor_finder(index, rels):
    precursors = []
    for i in range(len(rels['Index'])):
        if rels['Index'][i] == index:
            dummy = literal_eval(rels['Reagents'][i])
            for j in range(len(dummy)):
                precursors.append(dummy[j])
    return(precursors)

In [5]:
def find_component(node, rels):
    Product = node.tag
    path = (node.data).path
    indexes = index_finder(Product, rels, path)
    precursors = []
    for i in range(len(indexes)):
        precursors.append(precursor_finder(indexes[i], rels))
    return(indexes, precursors)
@contextmanager
def timeout(time):
    # Register a function to raise a TimeoutError on the signal.
    signal.signal(signal.SIGALRM, raise_timeout)
    # Schedule the signal to be sent after ``time``.
    signal.alarm(time)
    try:
        yield
    except TimeoutError:
        pass
    finally:
        # Unregister the signal so it won't be triggered
        # if the timeout is not reached.
        signal.signal(signal.SIGALRM, signal.SIG_IGN)
def raise_timeout(signum, frame):
    raise TimeoutError

In [6]:
def map_tree(Smiles, rels, max_trees, threshold_trees, time_limit):
    #Choose the molecule to generate Pathway
    #base_molecules = ['N', 'C=O', 'C(CO)=O', 'O'] #FormoseAmm
    #base_molecules = ['C=O', 'C(CO)=O', 'O'] #Formose
    #base_molecules = ['N', 'O', 'C(C(C(C(C(CO)O)O)O)O)=O'] #GlucoseAmm
    #base_molecules = ['C(C(C(C(C(CO)O)O)O)O)=O', 'O'] #Glucose
    #base_molecules = ['C(C(C)=O)(O)=O', 'O'] #PyruvicAcid
    #base_molecules = ['C=O', 'N', 'C#N', 'C#CC#N', 'O'] #UreyMiller
    base_molecules = ['NCC(=O)O', 'O=CC(O)C(O)C(O)C(O)C(O)', 'O'] #Maillard
    
    final_trees = []
    final_reactions = []
    final_energies = []
    
    all_trees = []
    tree_statuses = []
    tree1 = Tree()
    tree1.create_node(Smiles, 0, data=Molecule([Smiles]))
    all_trees.append(tree1)
    tree_statuses.append(False)
    reactions = [[]]
    complete = False
    early_complete = False
    threshold_reached = False
    with timeout(time_limit):
        while complete == False and early_complete == False and threshold_reached == False:
            for i in range(len(all_trees)):
                if tree_statuses[i] == False:
                    finished = False
                    current_nodes = all_trees[i].all_nodes()
                    current_depths = []
                    for j in range(len(current_nodes)):
                        current_depths.append(all_trees[i].depth(current_nodes[j]))
                    level_counter = max(current_depths)
                    node_counter = len(current_nodes)-1
                    while finished == False:
                        nodes = all_trees[i].all_nodes()
                        active_nodes = []
                        nodes_with_children = []
                        for k in range(len(nodes)):
                            try:     
                                temp = (all_trees[i].parent(nodes[k].identifier)).identifier
                            except:
                                temp = -1
                            if temp != -1:
                                nodes_with_children.append(temp)
                        for k in range(len(nodes)):
                            tag = nodes[k].tag
                            level = all_trees[i].depth(nodes[k])
                            if nodes[k].identifier not in nodes_with_children and tag not in base_molecules:
                                active_nodes.append(nodes[k])
                        if active_nodes == []:
                            finished = True
                            break
                        else:
                            for z in range(len(active_nodes)):
                                indexes, precursors = find_component(active_nodes[z], rels)
                                if len(precursors) == 0:
                                    all_trees[i] = 'NaN'
                                    finished = True
                                    break
                                else:
                                    product = active_nodes[z].identifier
                                    dummy = (active_nodes[z].data).path
                                    place = []
                                    num_trees = len(all_trees)
                                    for m in range(len(dummy)):
                                        place.append(dummy[m])
                                    if len(precursors) > 1:
                                        for p in range(1, len(precursors)):
                                            tree = tree_copier(all_trees[i], (num_trees*1000))
                                            dummy_product = product 
                                            tree_statuses.append(False)
                                            dummy_node_counter = num_trees*1000 + node_counter
                                            for q in range(len(precursors[p])):
                                                dummy_node_counter += 1
                                                tree.create_node(precursors[p][q], dummy_node_counter, parent=(product+num_trees*1000), data=Molecule(place + [precursors[p][q]])) 
                                            all_trees.append(tree)
                                            reactions.append(reactions[i] + [indexes[p]])
                                            num_trees+=1
                                    for n in range(len(precursors[0])):
                                        node_counter +=1
                                        all_trees[i].create_node(precursors[0][n], node_counter, parent=product, data=Molecule(place + [precursors[0][n]])) 
                                    reactions[i].append(indexes[0])
                if finished == True:
                    tree_statuses[i] = True
            #print(f'No. trees = {len(all_trees)}')
            num_complete_trees = 0
            for i in range(len(tree_statuses)):
                if tree_statuses[i] == True and all_trees[i] != 'NaN':
                    num_complete_trees += 1
            #print(f'No. complete trees = {num_complete_trees}')    
            final_trees = []
            final_reactions = []
            if num_complete_trees >= max_trees:
                early_complete = True
            else:
                dummy = True
                for i in range(len(tree_statuses)):
                    if tree_statuses[i] == False:
                        dummy = False
                if dummy == True:
                    complete = True
            if len(all_trees) > threshold_trees:
                threshold_reached = True
                break
        
        if early_complete == True or complete == True or threshold_reached == True:
            for i in range(len(all_trees)):
                if tree_statuses[i] == True and all_trees[i] != 'NaN':
                    final_trees.append(all_trees[i])
                    final_reactions.append(reactions[i])
            EnergyChanges = []
            ReactionIDs = []
            for i in range(len(rels['Index'])):
                EnergyChanges.append(rels['Energy Change'][i])
                ReactionIDs.append(rels['Index'][i])
            for i in range(len(final_trees)):
                dummy = 0
                for j in range(len(final_reactions[i])):
                    dummy += EnergyChanges[ReactionIDs.index(final_reactions[i][j])].round(2)
                final_energies.append(dummy)
    data = {'Tree':final_trees, 'Reaction IDs':final_reactions, 'Energy Change':final_energies}
    df = pd.DataFrame(data)
    return(final_trees, final_reactions, final_energies)

In [7]:
def gen_finder(string):
    string = list(string)
    return(int(string[1]))
def generations_counter(input_data, num_generations):
    gen_data = np.zeros(num_generations+1)
    value_data = np.zeros(num_generations+1)
    
    for i in range(len(gen_data)):
        gen_data[i] = i
        
    for i in range(len(input_data['Generation'])):
        dummy = split(input_data['Generation'][i])
        value = int(dummy[-1])
        value_data[value]+=1
    
    final_value_data = np.zeros(num_generations+1)
    final_value_data[0] = value_data[0]
    for i in range(1, len(value_data)):
        final_value_data[i] += value_data[i]
        for j in range(i):
            final_value_data[i] += value_data[j]
    
    return(gen_data, final_value_data)

In [8]:
def pathway_finder(matches_file, network, max_generation, num_generations, spontaneous, UseGenAppearsIn, UseRelsFile, MaxTrees, ThresholdTrees, TimeLimit): 
    matches_data = pd.read_csv(matches_file, sep='\t')
    gen_data, final_value_data = generations_counter(matches_data, num_generations)
    rels_data = []
    if UseGenAppearsIn == True:
        if spontaneous == False:
            rels_data.append(pd.read_csv(f'./ThermoRels/{network}G3_ThermoRels.tsv', sep='\t'))
        elif spontaneous == True:
            rels_data.append(pd.read_csv(f'./ThermoRels/{network}G3_Spontaneous_ThermoRels.tsv', sep='\t'))
    elif UseGenAppearsIn == False:
        if spontaneous == False:
            rels_data.append(pd.read_csv(f'MaillardRelsWithThermo.tsv', sep='\t'))
        elif spontaneous == True:
            rels_data.append(pd.read_csv(f'Spontaneous_MaillardG3RelsWithThermo.tsv', sep='\t'))
    all_smiles = []
    all_inchi = []
    all_gen = []
    all_trees = []
    all_energies = []
    all_reactions = []
    for i in range(int(final_value_data[max_generation])):
        print(i)
        gen = gen_finder(matches_data['Generation'][i])
        if UseGenAppearsIn==True:
            trees, reactions, energies = map_tree(matches_data['Smiles'][i], rels_data[gen-1], MaxTrees, ThresholdTrees, TimeLimit)
        elif UseGenAppearsIn==False:
            trees, reactions, energies = map_tree(matches_data['Smiles'][i], rels_data[-1], MaxTrees, ThresholdTrees, TimeLimit)
        all_trees.append(trees)
        all_energies.append(energies)
        all_reactions.append(reactions)
        all_smiles.append(matches_data['Smiles'][i])
        all_inchi.append(matches_data['Inchi'][i])
        all_gen.append(matches_data['Generation'][i]) 
        
        data = {'Generation':all_gen, 'Smiles':all_smiles, 'Inchi':all_inchi, 'Path Energies':all_energies, 'Reaction IDs':all_reactions} #'Pathways':all_trees
        df = pd.DataFrame(data)
        
        #This generates tsv file for each match that runs. 
        #This is to prevent loss of data in case when the Kernel dies.
        if spontaneous == False:
            df.to_csv(f'./backup/{i}{network}path.tsv', header=None, index=None, sep='\t', mode='a')
        elif spontaneous == True:
            df.to_csv(f'./backup/{i}{network}path_spontaneous.tsv', header=None, index=None, sep='\t', mode='a')
            
    return(df)

In [9]:
path = pathway_finder('MaillardMatchesG.tsv', 'Maillard', 4, 4, spontaneous=True, UseGenAppearsIn=False, UseRelsFile=3, MaxTrees=1, ThresholdTrees=10e10, TimeLimit=6000)

0
1


KeyboardInterrupt: 

In [ ]:
path.to_csv(f'Maillard_Pathway.tsv', header=None, index=None, sep='\t', mode='a')